<a href="https://colab.research.google.com/github/Dulavinya/Tea_defect_detection/blob/main/Yield_quality_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# 3. Install Ultr////////////'alytics ###

!pip install ultralytics


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:



!unzip -q "{zip_path}" -d ./FER

replace ./FER/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:


!pip install pyyaml

In [ ]:

import yaml

file_path = './FER/data.yaml'

with open(file_path, 'r') as file:
    data = yaml.safe_load(file)

print(data)


{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 1, 'names': ['leaves'], 'roboflow': {'workspace': 'children-and-adults-detection', 'project': 'leaves-detection-taesc', 'version': 2, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/children-and-adults-detection/leaves-detection-taesc/dataset/2'}}


In [ ]:

classes=data["names"]
print(classes)



In [ ]:


dict_classes={}
for i, j in enumerate(classes):
  dict_classes[i]=j
print(dict_classes)


updated_dict = {key: int(value) if value.isdigit() else value for key, value in dict_classes.items()}

print(updated_dict)

In [ ]:


%%writefile '.FER/growth_stage_detection_config.yaml'

path: './FER' # dataset root dir
train: train # train images (relative to 'path')
val: valid  # val images (relative to 'path')

names:
  0: 'high'
  1: 'law'
  2: 'medium'


In [ ]:
### 2. Define root directory ###
root_dir= './FER'

In [ ]:
import os

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8s.pt")  # load pre trained model

# Use the model
results = model.train(data=os.path.join(root_dir, "growth_stage_detection_config.yaml"), epochs=100,imgsz=640,patience=50,batch=0.70,device=0,
                      degrees=90)  # train the model


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("/content/runs/detect/train/weights/best.pt")  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
print(metrics.box.map)  # map50-95
print(metrics.box.map50)  # map50
print(metrics.box.map75)  # map75
print(metrics.box.maps) # a list contains map50-95 of each category

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("/content/runs/detect/train/weights/best.pt")

# Customize validation settings
validation_results = model.val(data=os.path.join(root_dir, "growth_stage_detection_config.yaml"), imgsz=640, batch=16, conf=0.25, iou=0.6, device="0")


In [ ]:

from ultralytics import YOLO

# Load a model
#model = YOLO("yolov8n.pt")  # pretrained YOLOv8n model
#model = YOLO("yolov8n.pt")
model_path="/content/runs/detect/train/weights/best.pt"

# Load a model
model = YOLO(model_path)  # load a custom model

# Run batched inference on a list of images
results = model("./FER/train/images/")
  # return a generator of Results objects


# Process results generator
for result in results:
    predict_out = result.boxes.data.tolist()
    print(predict_out)  # Boxes object for bounding box outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk


In [ ]:
results = model("./FER/test/images/")



# Process results generator
for result in results:
    predict_out = result.boxes.data.tolist()
    print(predict_out)  # Boxes object for bounding box outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

In [ ]:


### 5. Copy results ###
import subprocess

# Set environment variables and run the scp command

command = "scp -r /content/runs/detect/train '/content/drive/MyDrive/Work_space/Project/Harvest_pro/function2'"
process = subprocess.Popen(command, shell=True, env={"LC_ALL": "C.UTF-8", "LANG": "C.UTF-8"})
process.wait()